# Tools for NLP Part 1

### 1. Import koniecznych pakietów

Zaczynamy od importowania pakietu findspark, aby w dalszej kolejności umożliwić import Sparka i jego klas.

In [2]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

### 2. TF oraz IDF

In [17]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [18]:
sentenceData = spark.createDataFrame([
    (0.0, 'Hi I heard about Spark'),
    (0.0, 'I wish java could use case classes'),
    (1.0, 'Logistic,regression,models,are,neat')
], ['label', 'sentence'])

In [19]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish java could...|
|  1.0|Logistic,regressi...|
+-----+--------------------+



In [20]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [21]:
words_data = tokenizer.transform(sentenceData)

In [22]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish java could...|[i, wish, java, c...|
|  1.0|Logistic,regressi...|[logistic,regress...|
+-----+--------------------+--------------------+



In [23]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [24]:
featurized_data = hashing_tf.transform(words_data)

In [25]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [26]:
idf_model = idf.fit(featurized_data)

In [27]:
rescaled_data = idf_model.transform(featurized_data)

In [28]:
rescaled_data.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262144,[24417,49...|
|  0.0|(262144,[20719,24...|
|  1.0|(262144,[84234],[...|
+-----+--------------------+



### 3. CountVectorizer

In [29]:
from pyspark.ml.feature import CountVectorizer

In [30]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [31]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [32]:
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [33]:
model = cv.fit(df)

In [34]:
result = model.transform(df)

In [35]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

